# 토크나이제이션 (토큰화 : Tokenizaiton)
- 토큰화 : 문장을 단어 별로 나누는 것
- 수치화 : 토큰을 수치화 (벡터화) 하는 과정

## 벡터화

### 원 핫 인코딩

노미널 카테고리컬 데이터와 같은 방식으로 처리하고 <br>

각 단어가 인덱스이고 해당 단어는 1이고 그렇지 않은 단어는 0으로 표현 <br>

단어의 수가 많아질 수록 비효율적이다. sparce 하다. <br>

### 단어 임베딩

각 단어를 시퀀스화 하여 번호를 매기고 벡터화 하여 단어 간의 의미관계를 학습 <br>

토큰화 → 불용어 발라내기 → 벡터화 <br>

### IMDB를 이용한 영화평

In [1]:
from tensorflow.keras.datasets import imdb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.preprocessing import sequence
import torch

# 만개의 단어를 이용
max_features = 10000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [2]:
# 패딩 최대 길이
max_len = 200 

# 패딩 : 최대 길이보다 길경우 자르고, 짧을 경우 0으로 채운다.
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [3]:
# 모델 정의
good_bad_model = Sequential()

# 임베딩 과정 (벡터라이제이션) : 64개의 벡터값들을 초기는 랜덤하게 초기값을 한다.
# 이 초기값은 이를 학습하며 파라미터들이 업데이트 한다.-> 단어간의 의미관계 반영
# (총 데이터의 갯수, 200, 64) = (200 : 문서의 길이, 64: 벡터의 차원)
good_bad_model.add(Embedding(max_features, 64, input_length=max_len))

# 데이터별로 1D 로 만들어 각 노드별로 계산할수 있는 형태로 만든다.
# 총 2 차원 (데이터 포인트의 수, max_len * 임베딩 차원)
good_bad_model.add(Flatten())

# 학습 레이어 만들기
good_bad_model.add(Dense(32, activation = 'relu'))
good_bad_model.add(Dense(1, activation = 'sigmoid'))

# 컴파일 과정
good_bad_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 확인
good_bad_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 64)           640000    
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 32)                409632    
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,049,665
Trainable params: 1,049,665
Non-trainable params: 0
_________________________________________________________________


In [5]:
good_bad_model.fit(x_train,y_train,
                   epochs=10,
                   batch_size= 64,
                   validation_split = 0.2,
                   )

Epoch 1/10
313/313 [==============================] - 4s 6ms/step - loss: 0.4455 - accuracy: 0.7681 - val_loss: 0.2969 - val_accuracy: 0.8728
Epoch 2/10
313/313 [==============================] - 2s 7ms/step - loss: 0.1075 - accuracy: 0.9644 - val_loss: 0.3405 - val_accuracy: 0.8624
Epoch 3/10
313/313 [==============================] - 2s 6ms/step - loss: 0.0131 - accuracy: 0.9984 - val_loss: 0.3885 - val_accuracy: 0.8668
Epoch 4/10
313/313 [==============================] - 2s 6ms/step - loss: 0.0024 - accuracy: 0.9998 - val_loss: 0.4249 - val_accuracy: 0.8654
Epoch 5/10
313/313 [==============================] - 2s 6ms/step - loss: 9.6459e-04 - accuracy: 1.0000 - val_loss: 0.4443 - val_accuracy: 0.8668
Epoch 6/10
313/313 [==============================] - 2s 6ms/step - loss: 5.5311e-04 - accuracy: 1.0000 - val_loss: 0.4631 - val_accuracy: 0.8672
Epoch 7/10
313/313 [==============================] - 2s 6ms/step - loss: 3.6795e-04 - accuracy: 1.0000 - val_loss: 0.4781 - val_accuracy: 0

In [76]:
# 단어별 임베딩
# 0 ~ 9999 까지
good_bad_model.get_weights()[0][0]

array([-0.01089439, -0.01490348,  0.00304474, -0.01618349,  0.002664  ,
       -0.02977126,  0.00848474, -0.01083722,  0.00460668, -0.01055055,
        0.01836477,  0.01115856, -0.0081366 , -0.00030645,  0.0322329 ,
        0.01768994, -0.00036912,  0.01530584,  0.00323665,  0.00723391,
       -0.00680686, -0.00878318, -0.0235143 , -0.00043743, -0.00125735,
       -0.00958201,  0.00223411,  0.00856254,  0.01542935, -0.00662668,
        0.00643415, -0.00391647,  0.00358173,  0.00748805,  0.01662424,
        0.01216564,  0.00366573, -0.01816105,  0.01987887, -0.02183816,
       -0.0120273 ,  0.00837164, -0.00835055,  0.0019806 ,  0.0267897 ,
       -0.00565639,  0.00270927,  0.01105821, -0.00983608,  0.00172654,
       -0.01557806, -0.0063318 ,  0.00421915, -0.01677806, -0.00836237,
        0.0051714 ,  0.00444504,  0.01861977,  0.01789569,  0.00339429,
       -0.00541311,  0.02155936,  0.01253655,  0.01446076], dtype=float32)

In [86]:
pred = good_bad_model.predict(x_test)

782/782 [==============================] - 1s 1ms/step


In [92]:
pred_= [1 if i > 0.5 else 0 for i in pred]

In [97]:
correct = (pred_ == y_test)
correct.sum() / len(correct)

0.86764